In [2]:
#!pip install pyspark

In [3]:
from pyspark import SparkContext
from pyspark.sql import SparkSession


In [4]:
spark = SparkSession.builder.master('yarn-client').appName("mobod dz1").getOrCreate()

In [5]:
'''url_ratings = "https://raw.githubusercontent.com/HaykInanc/move_data/master/ratings.csv"
url_movies = "https://raw.githubusercontent.com/HaykInanc/move_data/master/movies.csv"
from pyspark import SparkFiles
spark.sparkContext.addFile(url_ratings)
spark.sparkContext.addFile(url_movies)

#df_ratings = spark.read.csv(SparkFiles.get("ratings.csv"), header=True, inferSchema= True)
#df_movies = spark.read.csv(SparkFiles.get("movies.csv"), header=True, inferSchema= True)'''
df_ratings = spark.read.csv("/data/mobod2020/movielens/ratings.csv", header=True, inferSchema= True)
df_movies = spark.read.csv("/data/mobod2020/movielens/movies.csv", header=True, inferSchema= True)


In [6]:
df_ratings.show()
df_movies.show()

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|     31|   2.5|1260759144|
|     1|   1029|   3.0|1260759179|
|     1|   1061|   3.0|1260759182|
|     1|   1129|   2.0|1260759185|
|     1|   1172|   4.0|1260759205|
|     1|   1263|   2.0|1260759151|
|     1|   1287|   2.0|1260759187|
|     1|   1293|   2.0|1260759148|
|     1|   1339|   3.5|1260759125|
|     1|   1343|   2.0|1260759131|
|     1|   1371|   2.5|1260759135|
|     1|   1405|   1.0|1260759203|
|     1|   1953|   4.0|1260759191|
|     1|   2105|   4.0|1260759139|
|     1|   2150|   3.0|1260759194|
|     1|   2193|   2.0|1260759198|
|     1|   2294|   2.0|1260759108|
|     1|   2455|   2.5|1260759113|
|     1|   2968|   1.0|1260759200|
|     1|   3671|   3.0|1260759117|
+------+-------+------+----------+
only showing top 20 rows

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+

In [7]:
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql.functions import udf
from pyspark.sql.window import Window

In [8]:
#Сортируем все записи в случайном порядке

windowSpec  = Window.partitionBy().orderBy("random")
df_train_test = (df_ratings.withColumn('random', F.rand())
                 .withColumn('rank', F.row_number().over(windowSpec))
                 .drop('random', 'timestamp')
                )
df_train_test.show()
df_train_test.count()


+------+-------+------+----+
|userId|movieId|rating|rank|
+------+-------+------+----+
|   177|   1784|   4.0|   1|
|   564|    926|   5.0|   2|
|   479|  54503|   5.0|   3|
|   561|    377|   3.0|   4|
|   353|    165|   3.5|   5|
|   633|    380|   2.0|   6|
|   124|   3066|   3.5|   7|
|   461|   3418|   4.0|   8|
|   152|   7502|   5.0|   9|
|   394|   3996|   2.5|  10|
|   609|    569|   1.0|  11|
|    93|    457|   4.0|  12|
|    23|  42418|   3.5|  13|
|   671|   2401|   4.0|  14|
|   236|   7943|   4.5|  15|
|   311|   7059|   3.5|  16|
|   212|  54190|   4.5|  17|
|   119|   1097|   4.0|  18|
|    56|  74416|   4.0|  19|
|   574|  53322|   3.0|  20|
+------+-------+------+----+
only showing top 20 rows



100004

In [9]:
#Выделяем обучающую выборку
df_train_without_group = df_train_test.where(F.col('rank') < df_train_test.count()*0.7).drop('rank')
df_train_without_group.show()
df_train_without_group.count()

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|   177|   1784|   4.0|
|   564|    926|   5.0|
|   479|  54503|   5.0|
|   561|    377|   3.0|
|   353|    165|   3.5|
|   633|    380|   2.0|
|   124|   3066|   3.5|
|   461|   3418|   4.0|
|   152|   7502|   5.0|
|   394|   3996|   2.5|
|   609|    569|   1.0|
|    93|    457|   4.0|
|    23|  42418|   3.5|
|   671|   2401|   4.0|
|   236|   7943|   4.5|
|   311|   7059|   3.5|
|   212|  54190|   4.5|
|   119|   1097|   4.0|
|    56|  74416|   4.0|
|   574|  53322|   3.0|
+------+-------+------+
only showing top 20 rows



70002

In [10]:
#выделяем тестирующую выборку
df_test = df_train_test.where(F.col('rank') >= df_train_test.count()*0.7).drop('rank')
df_test.show()
df_test.count()

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|   547|  71464|   3.5|
|   427| 103980|   3.5|
|   547|   8903|   3.5|
|   245|   1747|   4.0|
|   571|    546|   2.5|
|   358|    953|   5.0|
|   491|   1701|   5.0|
|   387|    428|   5.0|
|   294|   6358|   4.0|
|   412|    662|   1.0|
|   547| 101904|   3.5|
|    83|   3087|   3.5|
|   287| 102880|   0.5|
|   165|   8367|   2.0|
|   303|    508|   3.0|
|   472|   5010|   3.0|
|   494|   2947|   4.0|
|   664|   5574|   3.0|
|   551|   1085|   1.0|
|   471|   1080|   4.5|
+------+-------+------+
only showing top 20 rows



30002

In [11]:
#представляем рейтинги для каждого пользователя из обучающей выборки в виде словаря
df_train = (df_train_without_group.groupBy('userId')
            .agg(F.map_from_entries(F.collect_list(F.struct("movieId","rating"))))
            .withColumnRenamed('map_from_entries(collect_list(named_struct(NamePlaceholder(), movieId, NamePlaceholder(), rating)))', 'moviesList')
           )
df_train.show()

+------+--------------------+
|userId|          moviesList|
+------+--------------------+
|     1|[2968 -> 1.0, 140...|
|     2|[371 -> 3.0, 593 ...|
|     3|[27369 -> 3.5, 50...|
|     4|[1257 -> 5.0, 279...|
|     5|[30793 -> 3.5, 22...|
|     6|[8368 -> 3.5, 207...|
|     7|[1 -> 3.0, 198 ->...|
|     8|[4262 -> 4.0, 520...|
|     9|[2273 -> 3.0, 257...|
|    10|[1240 -> 4.0, 240...|
|    11|[96079 -> 4.0, 92...|
|    12|[3793 -> 5.0, 737...|
|    13|[6377 -> 4.5, 356...|
|    14|[2716 -> 3.0, 119...|
|    15|[3786 -> 2.0, 82 ...|
|    16|[50 -> 4.5, 4718 ...|
|    17|[288 -> 2.0, 1674...|
|    18|[141 -> 4.0, 36 -...|
|    19|[10 -> 3.0, 519 -...|
|    20|[1207 -> 4.0, 527...|
+------+--------------------+
only showing top 20 rows



In [12]:
def cos_distance(x1, x2):
  chisl = sum([x1[i] * x2[i] for i in x1.keys() if i in x2.keys()])
  znam1 = sum([x1[i] ** 2 for i in x1.keys()]) ** 0.5
  znam2 = sum([x2[i] ** 2 for i in x2.keys()]) ** 0.5
  return chisl / (znam1 * znam2)

cos_d_udf = F.udf(cos_distance, T.FloatType())


In [13]:
#считаем косинусное расстояние для каждой пары у которых не равны Id пользователей
#Для каждого пользователя из тестовой выборки получаем всех пользователей из обучающей
#Восстанавливаем рейтинг из обучающей выборки
#Оставляем двадцать самых похожих пользователей для каждого пользователя из тестовой
#
#На этом этапе теряются пользователи у которых нет не одной оценки в обучающей выборке
#А также фильмы которые в обучающей выборке никто не оценил
#Для них оценки посчитаем позже

df_ = df_train.withColumnRenamed('userId', 'userId2').withColumnRenamed('moviesList', 'moviesList2')

windowSpec  = Window.partitionBy('userId', 'movieId').orderBy(F.col("cos_d").desc())

df_user_cos_dist = (df_train.crossJoin(df_)
                    .withColumn('cos_d', cos_d_udf(F.col('moviesList'), F.col('moviesList2')))
                    .where(F.col('userId')!= F.col('userId2'))
                    .drop('moviesList')
                    .join(df_test, 'userId', 'inner')
                    .withColumn('get_rating', F.col('moviesList2')[F.col('movieId')])
                    .where(F.col('get_rating').isNotNull())
                    .drop('moviesList2')
                    .withColumn('rank', F.row_number().over(windowSpec))
                    .where(F.col('rank') <= 20)
                    .drop('rank')
                )
df_user_cos_dist.show()

+------+-------+-----------+-------+------+----------+
|userId|userId2|      cos_d|movieId|rating|get_rating|
+------+-------+-----------+-------+------+----------+
|    15|    388| 0.28023785|     14|   2.5|       4.0|
|    15|    102| 0.25200295|     14|   2.5|       4.0|
|    15|     30| 0.24700521|     14|   2.5|       4.0|
|    15|     19| 0.20279224|     14|   2.5|       5.0|
|    15|    311|  0.1876002|     14|   2.5|       3.0|
|    15|    659| 0.14234966|     14|   2.5|       4.0|
|    15|    588| 0.11297533|     14|   2.5|       3.0|
|    15|     86|0.096564315|     14|   2.5|       4.0|
|    15|    390|0.086217634|     14|   2.5|       2.0|
|    15|    266|0.076672174|     14|   2.5|       4.0|
|    15|    383| 0.06875245|     14|   2.5|       3.0|
|    15|     70| 0.06832393|     14|   2.5|       3.0|
|    15|    490|0.061814424|     14|   2.5|       3.0|
|    15|    284|0.060495175|     14|   2.5|       3.0|
|    15|    459|0.045762543|     14|   2.5|       4.0|
|    15|  

In [14]:
#получаем рейтинг исходя из косинусного расстояния
df_rating2 = (df_user_cos_dist.withColumn('umn', F.col('cos_d') * F.col('get_rating'))
          .groupBy('userId', 'movieId')
          .agg(F.sum(F.col('umn')), F.sum(F.col('cos_d')))
          .withColumn('rating2', F.col('sum(umn)') / F.col('sum(cos_d)'))
          .drop('sum(umn)', 'sum(cos_d)')
        )
df_rating2.show()

+------+-------+------------------+
|userId|movieId|           rating2|
+------+-------+------------------+
|    15|     14|3.7382511080398597|
|    15|   1199| 4.104534220200255|
|    15|   2060|   3.4447239173068|
|    15|   2387|2.6884732199212102|
|    15|   2594| 4.322899533099972|
|    15|   3527| 3.651757358898742|
|    15|   5346|               4.0|
|    17|   2010|3.8577420784861682|
|    18|     95|3.3703772857732517|
|    20|  64285|3.2191591708779668|
|    28|   1296| 4.014901619792919|
|    30|    590|3.4269470341414867|
|    30|   4710|               3.0|
|    30|   5961|               3.0|
|    47|     32| 3.890203444327218|
|    51|   2683|3.2430378070725476|
|    56|  71535| 3.427177162139523|
|    56|  79132| 3.959046381827814|
|    56| 103279|               3.0|
|    58|    940| 4.371149082025971|
+------+-------+------------------+
only showing top 20 rows



In [15]:
#для каждого фильма получаем средню оценку
df_sred = (df_train_without_group.groupBy('movieId')
           .agg(F.avg(F.col('rating')))
           .withColumnRenamed('avg(rating)', 'avg_r')
           .withColumnRenamed('movieId', 'movieId1')
          )
df_sred.show()

+--------+------------------+
|movieId1|             avg_r|
+--------+------------------+
|    1784|3.7916666666666665|
|     926| 4.425925925925926|
|   54503|3.8823529411764706|
|     377|             3.596|
|     165|  3.38659793814433|
|     380|3.5186567164179103|
|    3066|3.2142857142857144|
|    3418| 3.533333333333333|
|    7502|            4.3125|
|    3996|  3.77027027027027|
|     569|              2.25|
|     457|3.9044585987261144|
|   42418|              3.75|
|    2401|              3.75|
|    7943|               4.5|
|    7059|               3.5|
|   54190|               3.4|
|    1097| 3.730088495575221|
|   74416|              4.25|
|   53322|              3.25|
+--------+------------------+
only showing top 20 rows



In [16]:
#для каждого фильма получаем список жанров
df01 = df_movies.withColumn('razd_g', F.split(F.col('genres'), '\|')).drop('title', 'genres')
df01.show()

+-------+--------------------+
|movieId|              razd_g|
+-------+--------------------+
|      1|[Adventure, Anima...|
|      2|[Adventure, Child...|
|      3|   [Comedy, Romance]|
|      4|[Comedy, Drama, R...|
|      5|            [Comedy]|
|      6|[Action, Crime, T...|
|      7|   [Comedy, Romance]|
|      8|[Adventure, Child...|
|      9|            [Action]|
|     10|[Action, Adventur...|
|     11|[Comedy, Drama, R...|
|     12|    [Comedy, Horror]|
|     13|[Adventure, Anima...|
|     14|             [Drama]|
|     15|[Action, Adventur...|
|     16|      [Crime, Drama]|
|     17|    [Drama, Romance]|
|     18|            [Comedy]|
|     19|            [Comedy]|
|     20|[Action, Comedy, ...|
+-------+--------------------+
only showing top 20 rows



In [17]:
#соединяем жанры для фильмов про которых мы занем оценки со всеми фильмами
df_ = df01.join(df_sred, F.col('movieId') == F.col('movieId1'), 'inner').drop('avg_r', 'movieId1').withColumnRenamed('razd_g', 'razd_g2').withColumnRenamed('movieId', 'movieId2')
df_gen = df_.crossJoin(df01)
df_gen.show()

+--------+--------------------+-------+--------------------+
|movieId2|             razd_g2|movieId|              razd_g|
+--------+--------------------+-------+--------------------+
|    1784|[Comedy, Drama, R...|      1|[Adventure, Anima...|
|    1784|[Comedy, Drama, R...|      2|[Adventure, Child...|
|    1784|[Comedy, Drama, R...|      3|   [Comedy, Romance]|
|    1784|[Comedy, Drama, R...|      4|[Comedy, Drama, R...|
|    1784|[Comedy, Drama, R...|      5|            [Comedy]|
|    1784|[Comedy, Drama, R...|      6|[Action, Crime, T...|
|    1784|[Comedy, Drama, R...|      7|   [Comedy, Romance]|
|    1784|[Comedy, Drama, R...|      8|[Adventure, Child...|
|    1784|[Comedy, Drama, R...|      9|            [Action]|
|    1784|[Comedy, Drama, R...|     10|[Action, Adventur...|
|    1784|[Comedy, Drama, R...|     11|[Comedy, Drama, R...|
|    1784|[Comedy, Drama, R...|     12|    [Comedy, Horror]|
|    1784|[Comedy, Drama, R...|     13|[Adventure, Anima...|
|    1784|[Comedy, Drama

In [18]:
def cos_distance_gen(x1, x2):
  chisl = sum([1 for i in x1 if i in x2])
  znam1 = sum([1 for i in x1]) ** 0.5
  znam2 = sum([1 for i in x2]) ** 0.5
  return chisl / (znam1 * znam2)

cos_d_gen_udf = F.udf(cos_distance_gen, T.FloatType())

In [19]:
#Считаем косинусное расстояние между фильмами и оставляем 20 лучших для кадого фильма
windowSpec  = Window.partitionBy('movieId').orderBy(F.col("cos_d").desc())

df_cos_d_movie = (df_gen.withColumn('cos_d', cos_d_gen_udf(F.col('razd_g2'), F.col('razd_g')))
                  .where(F.col('movieId')!= F.col('movieId2'))
                  .drop('razd_g')
                  .withColumn('rank', F.row_number().over(windowSpec))
                  .where((F.col('rank') <= 20))
                  
         )
df_cos_d_movie.show()

+--------+-------+-------+-----+----+
|movieId2|razd_g2|movieId|cos_d|rank|
+--------+-------+-------+-----+----+
|     926|[Drama]|    148|  1.0|   1|
|   74416|[Drama]|    148|  1.0|   2|
|    1961|[Drama]|    148|  1.0|   3|
|    1103|[Drama]|    148|  1.0|   4|
|    3451|[Drama]|    148|  1.0|   5|
|    4211|[Drama]|    148|  1.0|   6|
|    5810|[Drama]|    148|  1.0|   7|
|    2757|[Drama]|    148|  1.0|   8|
|     147|[Drama]|    148|  1.0|   9|
|     354|[Drama]|    148|  1.0|  10|
|    3949|[Drama]|    148|  1.0|  11|
|    3098|[Drama]|    148|  1.0|  12|
|  106766|[Drama]|    148|  1.0|  13|
|    2188|[Drama]|    148|  1.0|  14|
|    3089|[Drama]|    148|  1.0|  15|
|    1293|[Drama]|    148|  1.0|  16|
|    1955|[Drama]|    148|  1.0|  17|
|     331|[Drama]|    148|  1.0|  18|
|    7234|[Drama]|    148|  1.0|  19|
|   27815|[Drama]|    148|  1.0|  20|
+--------+-------+-------+-----+----+
only showing top 20 rows



In [20]:
#получаем всех пользователей с фильмами которые никто не оценил в обучающей выборке
df_test_without_movie_rating = (
    df_test.withColumnRenamed('userId', 'userId1')
          .withColumnRenamed('rating', 'rating1')
          .drop('timestamp', 'rank')
          .join(df_train_without_group, 'movieId', 'left')
          .where(F.col('userId').isNull())
          .drop('timestamp', 'rank', 'userId', 'rating', 'timestamp', 'rank')
    )
df_test_without_movie_rating.show()

+-------+-------+-------+
|movieId|userId1|rating1|
+-------+-------+-------+
|   8903|    547|    3.5|
| 101904|    547|    3.5|
| 122490|    624|    1.5|
|  98908|    624|    1.5|
|   3025|    575|    3.0|
|  47404|     48|    4.0|
|    681|    492|    3.0|
|   7914|    468|    4.0|
|   4426|    547|    4.0|
|  67361|    624|    2.0|
|  26180|    547|    3.5|
|  31026|    294|    4.5|
| 107314|    563|    4.0|
|  55250|    624|    2.0|
| 111680|    624|    1.5|
|  25898|    529|    3.5|
|  33495|    353|    1.5|
| 113220|    547|    3.0|
|  91483|    522|    1.5|
|   5125|     30|    2.0|
+-------+-------+-------+
only showing top 20 rows



In [21]:
#получаем всех пользователей, которые не поставили не одной оценки в обучающей выборке
df_test_without_user_rating = (
    df_test.withColumnRenamed('movieId', 'movieId1')
          .withColumnRenamed('rating', 'rating1')
          .drop('timestamp', 'rank')
          .join(df_train_without_group, 'userId', 'left')
          .where(F.col('movieId').isNull())
          .drop('timestamp', 'rank', 'movieId', 'rating', 'timestamp', 'rank')
          .withColumnRenamed('userId', 'userId1')
          .withColumnRenamed('movieId1', 'movieId')
    )
df_test_without_user_rating.show()

+-------+-------+-------+
|userId1|movieId|rating1|
+-------+-------+-------+
+-------+-------+-------+



In [22]:
#объединяем эти две таблицы, удаляя дубликаты
df_without_rating = df_test_without_movie_rating.union(df_test_without_user_rating).distinct()
df_without_rating.show()

+-------+-------+-------+
|movieId|userId1|rating1|
+-------+-------+-------+
|   6797|    311|    3.5|
|   3879|     12|    5.0|
|   8504|    654|    4.5|
|   4837|    105|    3.5|
|   6186|     30|    2.0|
| 152079|     15|    1.5|
|  52617|     81|    5.0|
|   6022|    648|    4.0|
|  25965|    547|    2.0|
|   6143|    624|    2.0|
|  42900|    299|    5.0|
|  26810|    299|    4.5|
|   5672|    353|    0.5|
|  27544|    623|    4.0|
|  26025|    394|    0.5|
|   7958|    547|    3.5|
|   2219|    102|    4.0|
|   4684|    547|    0.5|
|   8999|    294|    4.0|
|   4565|    346|    5.0|
+-------+-------+-------+
only showing top 20 rows



In [23]:
#считаем рейтинг для фильмов
df_rating_movie2 = (
    df_sred.join(df_cos_d_movie, F.col('movieId1')==F.col('movieId2'), 'inner')
    .drop('razd_g2', 'movieId1')
    .withColumn('umn', F.col('cos_d') * F.col('avg_r'))
    .groupBy('movieId')
    .agg(F.sum(F.col('umn')), F.sum(F.col('cos_d')))
    .withColumn('rating2', F.col('sum(umn)') / F.col('sum(cos_d)'))
    .drop('sum(umn)', 'sum(cos_d)')
)
df_rating_movie2.show()

+-------+------------------+
|movieId|           rating2|
+-------+------------------+
|    148| 3.697825698629255|
|    463|3.6640213422133145|
|    471|3.1656652868467696|
|    496| 3.513623688392196|
|    833|3.1656652868467696|
|   1088| 3.073488930659983|
|   1238|3.1656652868467696|
|   1342|2.8778155623472967|
|   1580| 2.992637346312318|
|   1591|  3.45064905063004|
|   1645|3.5298667887660238|
|   1959|3.7732543768655042|
|   2122|2.8778155623472967|
|   2142| 3.547459462305487|
|   2366| 3.116999900582968|
|   2659|3.6957738095238093|
|   2866| 3.697825698629255|
|   3175| 3.260484363955824|
|   3794| 3.647112901879692|
|   3918|3.0683396464646466|
+-------+------------------+
only showing top 20 rows



In [24]:
#Считаем квадрат расхождения с реальными оценками
df_razn_rating = (
    df_without_rating.join(df_rating_movie2, 'movieId', 'inner')
    .withColumnRenamed('userId1', 'userId')
    .withColumnRenamed('rating1', 'rating')
    .withColumn('razn', F.pow(F.col('rating2') - F.col('rating'), 2))
)
df_razn_rating.show()

+-------+------+------+------------------+--------------------+
|movieId|userId|rating|           rating2|                razn|
+-------+------+------+------------------+--------------------+
|   1990|   564|   3.0|3.0683396464646466|0.004670307278912889|
|  50954|   255|   3.5|3.3065127241116414| 0.03743732593069778|
|  50954|   599|   4.0|3.3065127241116414| 0.48092460181905633|
|   1322|   564|   3.0|3.0683396464646466|0.004670307278912889|
|   1322|    95|   2.0|3.0683396464646466|  1.1413496002082062|
|   4391|   388|   4.0| 3.697825698629255| 0.09130930840889794|
|   8257|   468|   3.0|3.5957151248462216| 0.35487650997054937|
|  50792|   624|   2.5|3.7019077227772987|   1.444582174071712|
|  55498|   371|   3.5| 3.333824312166098|0.027614359227070378|
|    137|   564|   1.0|3.8306168581912465|   8.012391797876484|
|   4684|   547|   0.5|3.1656652868467696|    7.10577142149987|
|   5223|   598|   1.0| 3.647112901879692|   7.007206715297925|
|  26736|    73|   4.0|3.268982171225863

In [25]:
#счиатем квадрат расхождения для пользователей, которые сами ставили оценки, на фильмы которые кто-то оценил
df_result = (df_rating2.withColumnRenamed('userId', 'userId2')
          .withColumnRenamed('movieId', 'movieId2')
          .join(df_test, 
                (F.col('userId') == F.col('userId2')) & (F.col('movieId') == F.col('movieId2')), 
                'inner')
          .withColumn('razn', F.pow(F.col('rating2') - F.col('rating'), 2))
          .drop('userId2', 'movieId2', 'rating_list', 'timestamp', 'rank')
            )
df_result.show()

+------------------+------+-------+------+--------------------+
|           rating2|userId|movieId|rating|                razn|
+------------------+------+-------+------+--------------------+
|3.7382511080398597|    15|     14|   2.5|  1.5332658065619404|
| 4.104534220200255|    15|   1199|   1.0|   9.638132724394403|
|   3.4447239173068|    15|   2060|   2.0|  2.0872271972383056|
|2.6884732199212102|    15|   2387|   3.5|  0.6585757147850484|
| 4.322899533099972|    15|   2594|   4.0| 0.10426410847617977|
| 3.651757358898742|    15|   3527|   3.0|  0.4247876548786638|
|               4.0|    15|   5346|   3.5|                0.25|
|3.8577420784861682|    17|   2010|   5.0|   1.304753159261099|
|3.3703772857732517|    18|     95|   3.0| 0.13717933381676095|
|3.2191591708779668|    20|  64285|   5.0|  3.1713940586680507|
| 4.014901619792919|    28|   1296|   5.0|  0.9704188186866142|
|3.4269470341414867|    30|    590|   5.0|   2.474495633396265|
|               3.0|    30|   4710|   4.

In [26]:
#объединяем эти две таблицы
df_union_rating = df_razn_rating.union(df_result)
df_union_rating.show()

+--------+------+------+------------------+--------------------+
| movieId|userId|rating|           rating2|                razn|
+--------+------+------+------------------+--------------------+
|  1990.0|   564|   3.0|3.0683396464646466|0.004670307278912889|
| 50954.0|   599|   4.0|3.3065127241116414| 0.48092460181905633|
| 50954.0|   255|   3.5|3.3065127241116414| 0.03743732593069778|
|  1322.0|    95|   2.0|3.0683396464646466|  1.1413496002082062|
|  1322.0|   564|   3.0|3.0683396464646466|0.004670307278912889|
|  4391.0|   388|   4.0| 3.697825698629255| 0.09130930840889794|
|  8257.0|   468|   3.0|3.5957151248462216| 0.35487650997054937|
| 50792.0|   624|   2.5|3.7019077227772987|   1.444582174071712|
| 55498.0|   371|   3.5| 3.333824312166098|0.027614359227070378|
|   137.0|   564|   1.0|3.8306168581912465|   8.012391797876484|
|  4684.0|   547|   0.5|3.1656652868467696|    7.10577142149987|
|  5223.0|   598|   1.0| 3.647112901879692|   7.007206715297925|
| 26736.0|    73|   4.0|3

In [27]:
#высчитываем rmse
count_row = df_union_rating.count()
df_mser = df_result.select(F.sqrt(F.sum(F.col('razn'))/count_row)).alias('mser')
df_mser.show()

+-------------------------+
|SQRT((sum(razn) / 30002))|
+-------------------------+
|       0.9713346740199191|
+-------------------------+

